<a href="https://colab.research.google.com/github/PuruBhatnagar1197/Movie-multiclass-classification/blob/main/Movie_multiclass_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import requests
import re
import nltk
import string
import spacy
import textblob
from textblob import TextBlob
import tqdm
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
df=pd.DataFrame(columns=['movie_name','description','genere_id'])
for i in range(1,457):
    response=requests.get("https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page=" + str(i))
    print(i,response.status_code)
    for j in range(len(response.json()['results'])):
        movie_name=response.json()['results'][j]['original_title']
        description=response.json()['results'][j]['overview']
        genere_id=response.json()['results'][j]['genre_ids']
        df=df.append({'movie_name':movie_name,'description':description,'genere_id':genere_id},ignore_index=True)

df=df.rename(columns={'genere_id':'id'})

In [3]:
# df=df.rename(columns={'genere_id':'id'})
# df

In [ ]:
response_genre=requests.get("https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US")
print(response_genre.status_code)
df_genre=pd.DataFrame(columns=['id','genre'])
for i in range (1,len(response_genre.json()['genres'])):
    print(i)
    id=response_genre.json()['genres'][i]['id']
    name=response_genre.json()['genres'][i]['name']
    df_genre=df_genre.append({'id':id,'genre':name},ignore_index=True)


In [8]:
df_movie=df.explode('id')
df_movie_final=df_movie.merge(df_genre,on='id',how='left').dropna()

In [9]:
def remove_number_hyperlink(text):
    text_without_numbers = re.sub(r'\d+', '', text)
    text_without_hyperlinks = re.sub(r'https?://\S+|www\.\S+', '', text_without_numbers)
    return text_without_hyperlinks
def get_sentiment_vader(text):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score['neg'],sentiment_score['pos']

In [12]:
### text processing
ps=PorterStemmer()
stopwords=stopwords.words('english')
##1. lowering
df_movie_final['description']=df_movie_final['description'].str.lower()
print(df_movie_final['description'][0])
##2. removing punctions
df_movie_final['description']=df_movie_final['description'].str.translate(str.maketrans("","",string.punctuation))
print(df_movie_final['description'][0])
##3.basic spell checking and correction
# df_movie_final['description']=df_movie_final['description'].apply(lambda x: str(TextBlob(x).correct())) ### commented out because of time
##4. Stemming
df_movie_final['description']=df_movie_final['description'].apply(lambda x: ' '.join(ps.stem(word) for word in x.split()))
##5. Removal of stop words
df_movie_final['description']=df_movie_final['description'].apply(lambda x:' '.join(i for i in x.split() if i not in list(stopwords)))
##6. removal of numbers and hyperlinks
df_movie_final['description']=df_movie_final['description'].apply(lambda x: remove_number_hyperlink(x))

##7. tokenization
df_movie_final['description_tokenize']=df_movie_final['description'].apply(lambda x:word_tokenize(x))

##8. length of length of desc
tqdm.pandas(desc="for getting tone of the sentence")
df_movie_final['length_of_desc']=df_movie_final['description_tokenize'].progress_apply(lambda x: len(x))
df_movie_final=df_movie_final[df_movie_final['length_of_desc']>=1]



framed in the 1940s for the double murder of his wife and her lover, upstanding banker andy dufresne begins a new life at the shawshank prison, where he puts his accounting skills to work for an amoral warden. during his long stretch in prison, dufresne comes to be admired by the other inmates -- including an older prisoner named red -- for his integrity and unquenchable sense of hope.
framed in the 1940s for the double murder of his wife and her lover upstanding banker andy dufresne begins a new life at the shawshank prison where he puts his accounting skills to work for an amoral warden during his long stretch in prison dufresne comes to be admired by the other inmates  including an older prisoner named red  for his integrity and unquenchable sense of hope


for getting tone of the sentence:   0%|          | 0/21779 [00:00<?, ?it/s]

In [13]:
df_movie_final

,movie_name,description,id,genre,description_tokenize,length_of_desc
0,The Shawshank Redemption,frame doubl murder hi wife lover upstand bank...,18,Drama,"[frame, doubl, murder, hi, wife, lover, upstan...",41
1,The Shawshank Redemption,frame doubl murder hi wife lover upstand bank...,80,Crime,"[frame, doubl, murder, hi, wife, lover, upstan...",41
2,The Godfather,span year chronicl fiction italianamerican c...,18,Drama,"[span, year, chronicl, fiction, italianamerica...",32
3,The Godfather,span year chronicl fiction italianamerican c...,80,Crime,"[span, year, chronicl, fiction, italianamerica...",32
4,The Godfather Part II,continu saga corleon crime famili young vito c...,18,Drama,"[continu, saga, corleon, crime, famili, young,...",22
...,...,...,...,...,...,...
23914,The Adventures of Rocky & Bullwinkle,rocki bullwinkl live financ made rerun cartoon...,16,Animation,"[rocki, bullwinkl, live, financ, made, rerun, ...",34
23915,The Adventures of Rocky & Bullwinkle,rocki bullwinkl live financ made rerun cartoon...,35,Comedy,"[rocki, bullwinkl, live, financ, made, rerun, ...",34
23916,The Adventures of Rocky & Bullwinkle,rocki bullwinkl live financ made rerun cartoon...,14,Fantasy,"[rocki, bullwinkl, live, financ, made, rerun, ...",34
23917,Les Visiteurs: La Révolution,stuck corridor time godefroy de montmirail hi ...,35,Comedy,"[stuck, corridor, time, godefroy, de, montmira...",36


In [14]:
# ### Feature engineering   ### selected tfidf

# ## 1. Countvectorizer or bag of word
# ##1.1. countvectorize withoutputting ngram and binary and max features
# from sklearn.feature_extraction.text import CountVectorizer
# cnt=CountVectorizer()
# x=cnt.fit_transform(df_movie_final['description'])
# print("dimensionality of the vocabulary count vectorizer",len(cnt.get_feature_names_out()))
# print(x.toarray().shape)

# ##1.2. Countvectorizer with putting binary as true and max feature as 10
# cnt1=CountVectorizer(binary=True, max_features=20)  ### see how the max features got reduced
# x1=cnt1.fit_transform(df_movie_final['description'])
# print("dimensionality of the vocabulary with countvectorizer with max feature",len(cnt1.get_feature_names_out()))
# print(x1.toarray().shape)


# ##1.3. Countvectorizer with bigrams
# cnt3=CountVectorizer(ngram_range=(2,2))  ### see how the bigrams have increased the dimensionality
# x3=cnt3.fit_transform(df_movie_final['description'])
# print("dimensionality of the vocabulary with countvectorizer with ngrams",len(cnt3.get_feature_names_out()))
# print(x3.toarray().shape)


# ## 2. Tf-idf method
# from sklearn.feature_extraction.text import TfidfVectorizer
# tf_idf=TfidfVectorizer()
# x4=tf_idf.fit_transform(df_movie_final['description'])
# print("dimensionality of the vocabulary tfidf",len(tf_idf.get_feature_names_out()))
# print(len(tf_idf.idf_))

# ## 3. word2vec
# from gensim.models import Word2Vec
# word2vec1=Word2Vec(sentences=df_movie_final['description_tokenize'],window=10)
# print("dimensionality of the vocabulary",len(word2vec1.wv.index_to_key)) ### index_to_key is used to get the vocabulary

In [53]:
df_movie_final['genre'].unique()

array(['Drama', 'Crime', 'History', 'War', 'Animation', 'Family',
       'Fantasy', 'Comedy', 'Romance', 'Thriller', 'Adventure', 'Western',
       'Music', 'Horror', 'Mystery', 'Science Fiction', 'TV Movie'],
      dtype=object)

In [16]:
(df_movie_final['genre'].value_counts()/df_movie_final['genre'].shape[0])*100 ### a lot of imbalanced data

Drama              18.716136
Comedy             15.194233
Thriller           11.369272
Romance             7.075948
Adventure           7.043806
Crime               6.387180
Horror              6.051979
Science Fiction     5.110662
Fantasy             4.871889
Family              4.793829
Mystery             4.036183
Animation           3.843328
History             1.914776
War                 1.313252
Music               1.166315
Western             0.647442
TV Movie            0.463771
Name: genre, dtype: float64

In [17]:
### will build a model through tf_idf for simiplicity
### lets also make some knowledge based models

#feature-1:: getting positive nd negative score of amovie description
tqdm.pandas(desc="for getting tone of the sentence")
df_movie_final[['neg_score','pos_score']]=df_movie_final['description'].progress_apply(lambda x:pd.Series(get_sentiment_vader(x)))



for getting tone of the sentence:   0%|          | 0/21778 [00:00<?, ?it/s]

In [18]:
## feature-2: seeing the percentage share of most common word first and second across the movie description
first_most_common_word=[]
second_most_common_word=[]
value_first_comm=[]
value_second_comm=[]
for i, j in df_movie_final.iterrows():
    first_most_common_word.append(Counter(df_movie_final.loc[i,'description_tokenize']).most_common()[0][0])
    second_most_common_word.append(Counter(df_movie_final.loc[i,'description_tokenize']).most_common()[1][0])

for i, j in dict(Counter(first_most_common_word)).items():
    for element in first_most_common_word:
        if element==i:
            value_first_comm.append((j/Counter(first_most_common_word).most_common()[0][1]))
        else:
            continue

for i, j in dict(Counter(second_most_common_word)).items():
    for element in second_most_common_word:
        if element==i:
            value_second_comm.append((j/Counter(second_most_common_word).most_common()[0][1]))
        else:
            continue
df_movie_final['first_most_per']=value_first_comm
df_movie_final['second_most_per']=value_second_comm


In [20]:
from scipy.stats import yeojohnson,boxcox
df_movie_final['first_most_per']=list(boxcox(df_movie_final['first_most_per'])[0])
df_movie_final['second_most_per']=list(boxcox(df_movie_final['second_most_per'])[0])
# import seaborn as sns
#
# # sns.histplot(np.log(df_movie_final['first_most_per']/100))
# # sns.histplot(yeojohnson(df_movie_final['second_most_per']))
# # sns.histplot(boxcox(df_movie_final['first_most_per']))
# # sns.histplot(boxcox(df_movie_final['second_most_per']))

In [ ]:
### one of the way tot train and test split
train=pd.DataFrame()
test=pd.DataFrame()
from sklearn.model_selection import train_test_split
for i in list(df_movie_final['genre'].unique()):
    subset=df_movie_final[df_movie_final['genre']==i]
    train_subset,test_subset=train_test_split(subset,test_size=0.2,random_state=10)
    train=train.append(train_subset)
    test=test.append(test_subset)

train=train.sample(frac=1,random_state=10)
test=test.sample(frac=1,random_state=10)

In [ ]:
train

In [23]:
### using tf_idf to convert the text features into numbers
# from sklearn.feature_extraction import TfidfVectorizer
x_train=train.drop(columns=['genre','id','description_tokenize'])
y_train=train['genre']
x_test=test.drop(columns=['genre','id'])
y_test=test['genre']

In [24]:
x_train_numerical=x_train[['length_of_desc','neg_score','pos_score','first_most_per','second_most_per']]
x_test_numerical=x_test[['length_of_desc','neg_score','pos_score','first_most_per','second_most_per']]

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
# tfidf=TfidfVectorizer(max_features=1000)### the only drawback i this is that when you try to do a feature analysis it would be difficult to get the name of the feature so it is best to elminate this thing and use the below method
# train_with_vector_features=pd.DataFrame(tfidf.fit_transform(x_train['description']).toarray()+tfidf.fit_transform(x_train['movie_name']).toarray())  ### the only drawback i this is that when you try to do a feature analysis it would be difficult to get the name of the feature so it is best to elminate this thing and use the below method



### Train
tfidf_des=TfidfVectorizer(max_features=1000)
tfidf_movi=TfidfVectorizer(max_features=1000)
descrip_feature=tfidf_des.fit_transform(x_train['description'])
movie_feature=tfidf_movi.fit_transform(x_train['movie_name'])
description_array=hstack([descrip_feature,movie_feature])
columns_with_feature_name=list(tfidf_des.get_feature_names_out()) + list(tfidf_movi.get_feature_names_out())
from_vectorized_features=pd.DataFrame(description_array.toarray(),columns=columns_with_feature_name)
from_vectorized_features=from_vectorized_features.loc[:,~from_vectorized_features.columns.duplicated()].reset_index(drop=True).copy()
x_train=pd.concat([from_vectorized_features,x_train_numerical.reset_index(drop=True)],axis=1)
x_train

,abandon,abduct,abil,abl,abus,accept,accid,accident,accus,across,...,劇場版,劇場版ポケットモンスター,映画,疾風伝,신과함께,length_of_desc,neg_score,pos_score,first_most_per,second_most_per
0,0.0,0.0,0.0,0.0,0.0,0.285104,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27,0.132,0.150,-14.563372,-3.189013
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,15,0.000,0.162,-9.201202,-5.620579
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.199092,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,43,0.071,0.062,-11.771354,-6.494313
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,28,0.151,0.122,-2.244470,-2.037060
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32,0.158,0.129,-13.615391,-11.048931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17408,0.0,0.0,0.0,0.0,0.0,0.323381,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19,0.000,0.133,-4.905159,-6.659379
17409,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,24,0.106,0.288,-10.333462,-3.863571
17410,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10,0.514,0.000,-8.858377,-12.462792
17411,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20,0.000,0.333,-4.044265,-6.839391


In [26]:
#### test
desc_test_feat=tfidf_des.transform(x_test['description'])
movie_test_feat=tfidf_movi.transform(x_test['movie_name'])
des=hstack([desc_test_feat,movie_test_feat])
feat_test=list(tfidf_des.get_feature_names_out())+list(tfidf_movi.get_feature_names_out())
pre_final=pd.DataFrame(des.toarray(),columns=feat_test)

pre_final=pre_final.loc[:,~pre_final.columns.duplicated()].reset_index(drop=True)
x_test=pd.concat([pre_final,x_test_numerical.reset_index(drop=True)],axis=1)
x_test

,abandon,abduct,abil,abl,abus,accept,accid,accident,accus,across,...,劇場版,劇場版ポケットモンスター,映画,疾風伝,신과함께,length_of_desc,neg_score,pos_score,first_most_per,second_most_per
0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,51,0.160,0.118,-17.839346,-11.048931
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18,0.000,0.304,-5.599754,-2.989492
2,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,37,0.161,0.251,-17.839346,-5.722965
3,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,41,0.181,0.113,-5.399487,-12.462792
4,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,50,0.049,0.051,-11.771354,-7.037018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4360,0.0,0.0,0.0,0.0,0.0,0.0,0.12252,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,77,0.058,0.250,-12.876241,-15.179526
4361,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,45,0.000,0.247,-13.615391,-12.462792
4362,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13,0.000,0.000,-13.615391,-10.116900
4363,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,15,0.000,0.000,-3.645665,-12.462792


In [75]:
### encoding of y_variable
from sklearn.preprocessing import LabelEncoder
mb=LabelEncoder()
y_train_final=mb.fit_transform(y_train)
y_test_final=mb.transform(y_test)

In [33]:
#### modelling
### we will try with two models one is random forest and other is SVM linear model and will measure accuracy,precision and other things
### using onevsrest too
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
rf=RandomForestClassifier(class_weight='balanced')
clf_rf=OneVsRestClassifier(rf)

In [61]:
clf_rf.fit(x_train,y_train_final)

OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced'))

In [63]:
prediction=clf_rf.predict(x_test)

In [66]:
mb.inverse_transform(prediction)[1]

'Fantasy'

In [69]:
len(prediction)

4365

In [70]:
y_test.shape

(4365,)

In [78]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_final, prediction)

0.06849942726231387

In [79]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
svc=LinearSVC(class_weight='balanced')
clf_svc=OneVsRestClassifier(svc)
clf_svc.fit(x_train,y_train_final)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced'))

In [81]:
pred=clf_svc.predict(x_test)

In [83]:
mb.inverse_transform(pred)[0]

'Adventure'

In [87]:
from sklearn.metrics import accuracy_score,precision_score
print(accuracy_score(y_test_final, pred))
precision_score(y_test_final, pred,average='weighted')

0.15922107674684993


0.16540327681438768